In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os

N_STATES = 5   # ['action','close-avg','deal','amp_cover','vol_cover'] 
ACTIONS = ['None','Buy','Sell','OpenInteres','close-avg','deal','amp_cover','vol_cover'] #行為 空手、買進、賣出、持有、多單平倉、空單平倉
# ACTIONS = ['action','close-avg','deal','amp_cover','vol_cover']     # 探索者的可用动作
EPSILON = 0.9   # 贪婪度 greedy
ALPHA = 0.1     # 学习率
GAMMA = 0.9    # 奖励递减值
MAX_EPISODES = 13   # 最大回合数
FRESH_TIME = 0.3    # 移动间隔时间
global OpenInteres
OpenInteres = 0 # 是否持倉中
global priceAtPurchase #損益
priceAtPurchase = 0


In [2]:
direct=os.path.abspath('../data')
filelist = os.listdir('../data')
file = filelist[-1]
print(file)
dayticks = pd.read_csv(direct+'\\'+file,header=None,names=['ndatetime','nbid','nask','close','volume','deal'])
dayticks['ndatetime'] = pd.to_datetime(dayticks['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
dayticks.sort_values(by=['ndatetime'],ascending=True)
dayticks=dayticks[(dayticks.ndatetime.dt.hour>=8) & (dayticks.ndatetime.dt.hour<15)]
dayticks.index = dayticks.ndatetime
Candledf=dayticks['close'].resample('1min',closed='right').ohlc()
tmpdf=dayticks['volume'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
tmpdf=dayticks['deal'].resample('1min').sum()
Candledf=pd.concat([Candledf,tmpdf],axis=1)
del tmpdf
Candledf['dealcumsum']=Candledf['deal'].cumsum()
del Candledf['deal']
Candledf.rename(columns={'dealcumsum':'deal'},inplace=True)
Candledf=Candledf.rename_axis('ndatetime').reset_index()
Candledf['ndatetime'] = pd.to_datetime(Candledf['ndatetime'], format='%Y-%m-%d %H:%M:%S.%f')
Candledf.sort_values(by=['ndatetime'],ascending=True)
tmpline=Candledf.close.cumsum()
Candledf['Avg_line'] = tmpline.apply(lambda x: x/(tmpline[tmpline==x].index[0]+1))
del tmpline
Candledf['amplitude'] = Candledf.high - Candledf.low
Candledf['close_Avg'] = Candledf.close - Candledf.Avg_line
global CandleData
CandleData = Candledf[['ndatetime','close_Avg','deal','amplitude','volume','close']]
print(CandleData.tail())
global Total_Trade
Total_Trade = CandleData.shape[0]
Start_Run_Trade = 3



Ticks2021-12-14.txt
              ndatetime  close_Avg     deal  amplitude  volume  close
296 2021-12-14 13:40:00 -20.774411  17732.0          7   700.0  17605
297 2021-12-14 13:41:00 -24.691275  17774.0          5   424.0  17601
298 2021-12-14 13:42:00 -23.612040  17808.0          6   644.0  17602
299 2021-12-14 13:43:00 -22.536667  17808.0          4   498.0  17603
300 2021-12-14 13:44:00 -20.468439  17886.0          5  1304.0  17605


In [30]:
QT = pd.DataFrame(columns=ACTIONS,dtype=np.float32)
print(QT)
QT=QT.append(pd.Series([0]*len(ACTIONS),index=ACTIONS,name='state'))
print(QT.loc['state',QT.columns[:3]])

Empty DataFrame
Columns: [None, Buy, Sell, OpenInteres, close-avg, deal, amp_cover, vol_cover]
Index: []
None    0.0
Buy     0.0
Sell    0.0
Name: state, dtype: float64


In [ ]:
class QLearning:
    def __init__(self,actionslist=None,LearnRate=0.01,disscount_factory=0.9,e_greedy=0.1):
        self.actions = actionslist # action 列表
        self.lr = LearnRate #學習效率
        self.gamma = disscount_factory # 折扣因子
        self.espilon = e_greedy #貪婪度
        self.q_table = pd.DataFrame(columns=self.actions,dtype=np.float32) #建立空Q Table
        self.openInteres = 0
        
    def check_state_exist(self,state):
        if state not in self.q_table.index:
            self.q_table = self.q_table.append(pd.Series([0]*len(self.actions),index=self.q_table.columns,name=state))
    
    def choose_action(self,state):
        self.check_state_exist(state)
        #隨機選行為 Epsilon Greedy貪婪方法
        if np.random.uniform()>self.espilon:
            #隨機選 1個action
            action = np.random.choice(self.actions[:3])
        else: #取Q值最高的action
            state_action = self.q_table.loc[state,:3]
            state_action = state_action.reindex(np.random.permutation(state_action.index))
            # 取當下最大Q值
            action = state_action.idxmax()
        return action
    # Q Table 學習
    def learn(self,s,a,r,s_): #state,action,result,s_(下個state)
        #檢查下個 state
        self.check_state_exist(s_) #檢查s_ 是否在q_table
        # Q(S,A) <- Q(S,A)+a*[R+v*max(Q(S',a)-Q(S,A))]
        q_predict = self.q_table[s,a] #根據Q表 得到 預估值 predict
        #q_target 現實值
        if s_ != 'terminal': #下一步 不是最終
            q_target = r + self.gamma * self.q_table[s_,:].max() 
        else: # 下一步終結
            q_target = r 
        # 更新Q表
        self.q_table.loc[s,a] += self.lr * (q_target - q_predict)


In [ ]:
QT = QLearning(actionslist=ACTIONS)
QT.choose_action(1)


In [ ]:
# https://blog.csdn.net/joinquantdata/article/details/87254611?spm=1001.2101.3001.6650.6&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-6.no_search_link
class Market:
    def __init__(self,data) -> None:
        self.action_list = ['Buy','Sell','Wait'] # 買進、賣出、觀望
        self.actions = len(self.action_list)
        self.data = data # 1分鐘 K線資料
        self.idx = 10 #預設開始跑策略的K線
        self.time = self.data.at[self.idx,'ndatetime'].time().strftime('%H:%M')
        self.OpenInterest = 0

    def get_state(self,idx):
        if idx > self.idx:
            state = self.data.loc[idx,['close_Avg','dealcumsum','amplitude','volume','close']]
        return state

    def step(self,action):
